# Seção 14 - Regressão Linear (Desafio_Auto Imports Database)
Para esta seção, usarei a base de dados descrita abaixo:

1. Title: 1985 Auto Imports Database

2. Source Information:
   
   -- Creator/Donor: Jeffrey C. Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
   
   -- Date: 19 May 1987
   
   -- Sources:
     
     1) 1985 Model Import Car and Truck Specifications, 1985 Ward's Automotive Yearbook.
     
     2) Personal Auto Manuals, Insurance Services Office, 160 Water Street, New York, NY 10038 
     
     3) Insurance Collision Report, Insurance Institute for Highway Safety, Watergate 600, Washington, DC 20037
        
A base acima citadada pode ser acessada no link: https://archive.ics.uci.edu/ml/datasets/Automobile.

In [1]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from plotting import (multiple_histograms_plot,
                      bar_plot_with_categorical,
                      plot_confusion_matrix,
                      plot_confusion_matrix_2,
                      plot_roc)

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Lidar com validação de modelos.
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [2]:
nomeDaBase = "../bases/carros/imports-85.csv"
df = pd.read_csv(nomeDaBase, sep=",", encoding="utf-8")

In [3]:
df.sample(5).T

160     113        136      89      97
symboling          0       0       3          1       1     
normalized-losses  91      ?       150        128     103   
make               toyota  peugot  saab       nissan  nissan
fuel-type          gas     gas     gas        gas     gas   
aspiration         std     std     turbo      std     std   
num-of-doors       four    four    two        two     four  
body-style         sedan   wagon   hatchback  sedan   wagon 
drive-wheels       fwd     rwd     fwd        fwd     fwd   
engine-location    front   front   front      front   front 
wheel-base         95.7    114.2   99.1       94.5    94.5  
length             166.3   198.9   186.6      165.3   170.2 
width              64.4    68.4    66.5       63.8    63.8  
height             53      56.7    56.1       54.5    53.5  
curb-weight        2094    3285    2808       1889    2037  
engine-type        ohc     l       dohc       ohc     ohc   
num-of-cylinders   four    four    four       four    four  
engine-size        98      120     121        97      97    
fuel-system        2bbl    mpfi    mpfi       2bbl    2bbl  
bore               3.19    3.46    3.54       3.15    3.15  
stroke             3.03    2.19    3.07       3.29    3.29  
compression-ratio  9       8.4     9          9.4     9.4   
horsepower         70      95      160        69      69    
peak-rpm           4800    5000    5500       5200    5200  
city-mpg           38      19      19         31      31    
highway-mpg        47      24      26         37      37    
price              7738    16695   18150      5499    7999

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    205 non-null object
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         205 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 205 non-null object
stroke               205 non-null object
compression-ratio    205 non-null float64
horsepower           205 non-nul

In [5]:
df.describe()

symboling  wheel-base      length       width      height  \
count  205.000000  205.000000  205.000000  205.000000  205.000000   
mean   0.834146    98.756585   174.049268  65.907805   53.724878    
std    1.245307    6.021776    12.337289   2.145204    2.443522     
min   -2.000000    86.600000   141.100000  60.300000   47.800000    
25%    0.000000    94.500000   166.300000  64.100000   52.000000    
50%    1.000000    97.000000   173.200000  65.500000   54.100000    
75%    2.000000    102.400000  183.100000  66.900000   55.500000    
max    3.000000    120.900000  208.100000  72.300000   59.800000    

       curb-weight  engine-size  compression-ratio    city-mpg  highway-mpg  
count  205.000000   205.000000   205.000000         205.000000  205.000000   
mean   2555.565854  126.907317   10.142537          25.219512   30.751220    
std    520.680204   41.642693    3.972040           6.542142    6.886443     
min    1488.000000  61.000000    7.000000           13.000000   16.000000    
25%    2145.000000  97.000000    8.600000           19.000000   25.000000    
50%    2414.000000  120.000000   9.000000           24.000000   30.000000    
75%    2935.000000  141.000000   9.400000           30.000000   34.000000    
max    4066.000000  326.000000   23.000000          49.000000   54.000000

### Observação:
Na base os valores com a string "?" indicam *nulls*. Eu creio que isso pode nos levar a conclusões errôneas como estimar errôneamente a quantidade de *nulls*. Vou substituir os "?" por "nulls".

In [6]:
df.replace(to_replace="?", value=np.nan, inplace=True)

**ATENÇÃO:** Perceba que agora alguns dos atributos possuem nulos.

Agora, vou transformar os atributos que ainda estão como objects em floats.

In [7]:
lista_converter = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

In [8]:
for item in lista_converter:
    df[item] = df[item].astype("float64")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    164 non-null float64
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         203 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression-ratio    205 non-null float64
horsepower           203 non-

In [10]:
# state_dummies = pd.get_dummies(x_train["addr_state"],drop_first=True)
# x_train = pd.concat([x_train,state_dummies],axis=1)

df_teste = df

In [11]:
lista_atributos_categoricos = ["make", "fuel-type", "aspiration", "num-of-doors",
                               "body-style", "drive-wheels", "engine-location", "engine-type",
                               "num-of-cylinders", "fuel-system"]

In [12]:
state_dummies = pd.get_dummies(df_teste[lista_atributos_categoricos])
df_teste.drop(lista_atributos_categoricos, axis=1, inplace=True)

df_teste = pd.concat([df_teste, state_dummies], axis=1)

df_teste.head()

## Ponto de Atenção
Nesse ponto vou tenho duas alternativas:

1. Remover os registros que possuem nulos; ou
2. Substituir os valores nulos por outros valores coerentes com a distribuição observada naquele atributo.

Vou comparar os resultados das duas abordagens.

### 1) Removendo os registros que possuem nulos

In [15]:
df_teste_semNulos = df_teste.dropna(axis=0, how="any")

In [16]:
queda = 100*(len(df_teste)-len(df_teste_semNulos))/len(df_teste)
print("Queda de {:.4}% de registros, passamos de {} para {} registros.".
      format(queda, len(df_teste), len(df_teste_semNulos)))

Queda de 21.95% de registros, passamos de 205 para 160 registros.


**A queda no número de registros foi elevada.** Isso indica que a abordagem de remoção de registros pode não ser boa. Vamos continuar com essa abordagem e comparar com a segunda abordagem.

In [17]:
x = df_teste_semNulos.drop("price", axis=1)
x = x.iloc[:,:].values
y = df_teste_semNulos.price.values

print(type(x), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [49]:
n = x_test.shape[0]
k = x_test.shape[1]

print(n, k)

32 75


In [63]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("RMSE: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("MAE: {:.2f}".format(mean_absolute_error(y_test, y_pred)))
# print("MAPE:, ", mean_absolute_percentage_error(y_test, y_pred))
print("R2: {:.2f}".format(r2_score(y_test, y_pred)))

################################################################################
# Cálculo do R2 ajustado:
def r2_ajustado(x_train, y_test, y_pred):
    """
    r2_ajustado retorna o R² Ajustado e recebe como argumento as séries com o
    valor alvo teste e o predito.
    """
    n = x_train.shape[0]
    k = x_train.shape[1]
    return (1-((n-1)/(n-(k+1)))*(1-r2_score(y_test, y_pred)))
################################################################################

print("R2 Ajustado: {:.2f}".format(r2_ajustado(x_train, y_test, y_pred)))
print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred)))

RMSE: 1671.41
MAE: 1237.89
R2: 0.79
R2 Ajustado: 0.48
MSE: 2793613.49


**Testando um registro aleatório:**

In [62]:
from random import seed, randint
seed = 42
aleatorio = randint(0,len(x))
resgitro_teste = x[aleatorio]
print("Registro n.º: {}\n".format(aleatorio))

y_pred_teste = regressor.predict(resgitro_teste.reshape(1, -1))

print("Valor real: {:.2f}".format(y[aleatorio]))
print("Valor predito: {:.2f}".format(y_pred_teste[0]))

diferença = abs(y[aleatorio]-y_pred_teste[0])
print("Diferença de {:.2f} ou {:.2f}%".format(diferença, 100*diferença/y[aleatorio]))

Registro n.º: 21

Valor real: 6855.00
Valor predito: 6182.03
Diferença de 672.97 ou 9.82%


# Conclusão
O modelo de regressão liner múltipla obtido possui boa aderência ao fenômeno (R² = 0.79).